# GETTING INFORMATION FROM AUTHORS

In [1]:
import json, os
import datetime
from pydantic import HttpUrl
import pandas as pd
from tqdm import tqdm

from pnytter import Pnytter

## 1. Load the Authors list

In [2]:
df_retweets = pd.read_csv('202503192227-RETWEETS_ETIQUETADOS_CONTAINS_HATE_TRUE.csv', index_col=None)
df_retweets = df_retweets.rename(columns={'count_star()':'ocurrencias'})

In [10]:
df_retweets.head()

,tweet_id,author,ocurrencias
0,1795781220925669620,963492493458329602,2
1,1795383660088074396,796158301788667908,2
2,1795168528464540035,482147864,2
3,1798852393096130705,1621152552258801672,2
4,1801196737324843027,1665677569671168002,2


In [4]:
df_retweets.shape

(1776, 3)

## 2. Get the Author's information

In [12]:
# The Pnytter object needs at least 1 Nitter instance to work, but these can be added after initialization
pnytter = Pnytter(
  #nitter_instances=["http://localhost:8080"]
  nitter_instances=["https://xcancel.com", "https://nitter.privacydev.net", "https://lightbrd.com"]
)

In [6]:
if os.path.exists('retweets_temp.csv'):
    df_anterior = pd.read_csv('retweets_temp.csv')

In [13]:
cont = 0

data = {
    'tweet_id': [],
    'author': [],
    'author_id': [],
    'created_at': [],
    'text': [],
    'retweets_count': [], 
    'comments_count': [],
    'likes_count': [],
    'quotes_count': []
}

error = {
    'tweet_id': [],
}

for row in tqdm(df_retweets.itertuples(), total=len(df_retweets), desc='Buscando Retweets'):
    
    # check if the tweet was already processed
    if 'df_anterior' in locals():
        if row.tweet_id in df_anterior.tweet_id.values:
            cont += 1
            continue

    try:
        tweet = pnytter.get_tweet(row.tweet_id)
    except Exception as e:
        cont += 1
        print(f"Error al obtener el tweet {row.tweet_id}: {e}")
        error['tweet_id'].append(row.tweet_id)
        continue

    try: 
        data['author'].append(tweet.author) 
        data['created_at'].append(tweet.created_on)
        data['text'].append(tweet.text)
        data['retweets_count'].append(tweet.stats.retweets)
        data['comments_count'].append(tweet.stats.comments)
        data['likes_count'].append(tweet.stats.likes)
        data['quotes_count'].append(tweet.stats.quotes)
        data['tweet_id'].append(row.tweet_id)
        data['author_id'].append(row.author)

    except AttributeError:
        error['tweet_id'].append(row.tweet_id)

    cont += 1

    if cont == 100:
        cont = 0       
        df = pd.DataFrame(data)
        df_error = pd.DataFrame(error)

        # Verificar si los archivos existen antes de eliminarlos
        if os.path.exists('retweets_temp.csv'):
            os.remove('retweets_temp.csv')
        if os.path.exists('errors_temp.csv'):
            os.remove('errors_temp.csv')

        df.to_csv('retweets_temp.csv', index=False)
        df_error.to_csv('errors_temp.csv', index=False)


df = pd.DataFrame(data)
df_error = pd.DataFrame(error)
df.to_csv('retweets.csv', index=False)
df_error.to_csv('errors.csv', index=False)
print("Archivos retweets.csv y errors.csv generados correctamente.")


Buscando Retweets:   1%|▏         | 23/1776 [00:00<00:41, 41.82it/s]

Error al obtener el tweet 1804615146939003047: 429 Client Error: Too Many Requests for url: https://nitter.privacydev.net/_/status/1804615146939003047


Buscando Retweets:   2%|▏         | 28/1776 [00:00<00:54, 31.85it/s]

Error al obtener el tweet 1795424189316301071: 429 Client Error: Too Many Requests for url: https://nitter.privacydev.net/_/status/1795424189316301071


Buscando Retweets:   6%|▌         | 104/1776 [00:01<00:15, 107.29it/s]

...

Error al obtener el tweet 1801522541019742524: 429 Client Error: Too Many Requests for url: https://nitter.privacydev.net/_/status/1801522541019742524


Buscando Retweets: 100%|█████████▉| 1774/1776 [02:03<00:00,  2.45it/s]

Error al obtener el tweet 1800251230419357813: 403 Client Error: Forbidden for url: https://xcancel.com/_/status/1800251230419357813


Buscando Retweets: 100%|██████████| 1776/1776 [02:04<00:00, 14.23it/s]

Error al obtener el tweet 1801279622111191227: 429 Client Error: Too Many Requests for url: https://nitter.privacydev.net/_/status/1801279622111191227
Archivos retweets.csv y errors.csv generados correctamente.


# !!! hacer un concat de retweets_temp y retweets

In [ ]:
df = pd.DataFrame(data)
df.to_csv('tweets.csv', index=False)
df.shape

In [ ]:
len(data['tweet_id'])

In [ ]:
len(data['author'])

### 2.1 Primero deberemos obtener los datos del Retweet

A través de los datos del retweet, obtenemos los datos del usuario, e información estadística del alcance del retweet.

In [ ]:
# The Pnytter object needs at least 1 Nitter instance to work, but these can be added after initialization
pnytter = Pnytter(
  nitter_instances=["http://localhost:8080"]
)

In [ ]:
def get_retweet(tweet_id):
    tweet = pnytter.get_tweet(tweet_id)
    data = {
        'tweet_id': [],
        'author': [],
        'created_at': [],
        'text': [],
        'retweets_count': [],
        'comments_count': [],
        'likes_count': [],
        'quotes_count': []
    }

    data['tweet_id'].append(tweet.tweet_id)
    data['author'].append(tweet.author) 
    data['created_at'].append(tweet.created_on)
    data['text'].append(tweet.text)
    data['retweets_count'].append(tweet.stats.retweets)
    data['comments_count'].append(tweet.stats.comments)
    data['likes_count'].append(tweet.stats.likes)
    data['quotes_count'].append(tweet.stats.quotes)

    return data

In [ ]:
user = json.loads(f"""{'id': None,
 'username': 'agustinpicag',
 'fullname': 'Agustin Picapiedra G',
 'biography': '',
 'verified': True,
 'joined_datetime': datetime.datetime(2014, 12, 29, 15, 57, tzinfo=datetime.timezone.utc),
 'stats': {'tweets': 632, 'following': 305, 'followers': 45, 'likes': 39245},
 'pictures': {'profile': {'twitter_url': HttpUrl('https://pbs.twimg.com/profile_images/1021677838737592321/hFfXu_e_.jpg'),
                          'nitter_path': '/pic/pbs.twimg.com%2Fprofile_images%2F1021677838737592321%2FhFfXu_e_.jpg'},
              'banner': None}}""")

In [ ]:
import pydantic
pydantic.__version__